In [13]:
import svgBlockLib

from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False, show_parent=False)

enable_replay(False, False)
show_object = replay

cv = open_viewer("3dViewer", anchor="right", height=1024)


Enabling jupyter_cadquery replay


In [14]:
blk = svgBlockLib.blkLibrary()

In [15]:
#Import from file. Could be web or others in the future
#fileName = 'noun-gutenberg-press-86670-cleaned.svg'
#fileName = 'noun-houseplants-6613684.svg'
#fileName = 'noun-houseplants-886781-edited.svg'
fileName = 'noun-ornaments-714185-cleaned.svg'
blk.readSVGFromFile(fileName)

#Parse using svgpathtools and 'return' paths and attribs.
#Could probably have some checks to make sure paths exist and are closed?
blk.parseSVG()

#See how many paths were parsed
print('Paths Found: %s' % format(blk.pathCount))

#Skip paths. Helpful for testing or omitting paths. 
#Needs to be tested
#blk.skipPathNumber.append([100,200])

#Do the actual translation from 2D to 3D
blk.translate2Dto3D()

blk.neckHeight = 2

blk.xLenAdj = 1.25
blk.yLenAdj = 1.25
blk.buildBody()

Paths Found: 1


In [16]:
show_object(blk.base)

Use the multi select box below to select one or more steps you want to examine
